In [48]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [49]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


In [50]:
datastore = [json.loads(line) for line in open('Sarcasm_Headlines_Dataset.json', 'r')]
    
    
sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [51]:
tokenizer = Tokenizer(oov_token='<00V>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

In [52]:
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding='post')



In [53]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [54]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length,
                               padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length,
                              padding=padding_type, truncating=trunc_type)






In [60]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [61]:
# embeding with nueral net

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [62]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [63]:
# training the model
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
625/625 - 2s - loss: 0.6685 - accuracy: 0.5852 - val_loss: 0.5959 - val_accuracy: 0.7459
Epoch 2/30
625/625 - 2s - loss: 0.4212 - accuracy: 0.8304 - val_loss: 0.3876 - val_accuracy: 0.8265
Epoch 3/30
625/625 - 1s - loss: 0.3048 - accuracy: 0.8784 - val_loss: 0.3507 - val_accuracy: 0.8490
Epoch 4/30
625/625 - 2s - loss: 0.2542 - accuracy: 0.8986 - val_loss: 0.3415 - val_accuracy: 0.8557
Epoch 5/30
625/625 - 2s - loss: 0.2182 - accuracy: 0.9151 - val_loss: 0.3428 - val_accuracy: 0.8565
Epoch 6/30
625/625 - 1s - loss: 0.1906 - accuracy: 0.9266 - val_loss: 0.3715 - val_accuracy: 0.8432
Epoch 7/30
625/625 - 1s - loss: 0.1712 - accuracy: 0.9344 - val_loss: 0.3644 - val_accuracy: 0.8535
Epoch 8/30
625/625 - 1s - loss: 0.1510 - accuracy: 0.9460 - val_loss: 0.3820 - val_accuracy: 0.8530
Epoch 9/30
625/625 - 1s - loss: 0.1362 - accuracy: 0.9512 - val_loss: 0.4026 - val_accuracy: 0.8505
Epoch 10/30
625/625 - 1s - loss: 0.1225 - accuracy: 0.9575 - val_loss: 0.4229 - val_accuracy: 0.8495

In [68]:
# predict on new sentences

sentence = [
    'granny starting to fear spiders in the garden might be real',
    'the weather today is bright and sunny']

sequences = tokenizer.texts_to_sequences(sentence)

padded = pad_sequences(sequences, maxlen=max_length,
                      padding=padding_type,
                      truncating=trunc_type)

print(model.predict(padded))

[[9.9249256e-01]
 [5.1118510e-05]]


In [77]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [78]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 64)          640000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 128)         66048     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_14 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 65        
Total params: 751,489
Trainable params: 751,489
Non-trainable params: 0
_________________________________________________________________
